In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetB0
import matplotlib.pyplot as plt

2023-03-03 20:19:01.199259: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-03 20:19:01.859221: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-03 20:19:01.859288: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-03 20:19:01.859296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

IMG_SIZE = 128;
NUM_CLASSES = 10;
batch_size = 16;
size = (IMG_SIZE,IMG_SIZE);

(ds_train, ds_test), ds_info = tfds.load(
    "imagenette", split=['train','validation'], with_info=True, as_supervised=True
)

ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

# ds_train = ds_train.map(lambda image, label: (tf.expand_dims(tf.image.resize(image, size),axis=0), label))
# ds_test = ds_test.map(lambda image, label: (tf.expand_dims(tf.image.resize(image, size),axis=0), label))


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [9]:
img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = img_augmentation(inputs)
outputs = EfficientNetB0(
    include_top=True,
    weights=None,
    classes=10,
    input_shape=(128,128,3),
    
)(x)
model = tf.keras.Model(inputs, outputs)
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 img_augmentation (Sequentia  (None, 128, 128, 3)      0         
 l)                                                              
                                                                 
 efficientnetb0 (Functional)  (None, 10)               4062381   
                                                                 
Total params: 4,062,381
Trainable params: 4,020,358
Non-trainable params: 42,023
_________________________________________________________________


In [11]:
epochs = 5  # @param {type: "slider", min:10, max:100}
hist = model.fit(ds_train, epochs=epochs, validation_data=ds_test, verbose=2)

Epoch 1/5


2023-03-03 20:21:03.569233: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


591/591 - 129s - loss: 2.7339 - accuracy: 0.1935 - val_loss: 2.2053 - val_accuracy: 0.2304 - 129s/epoch - 219ms/step
Epoch 2/5
591/591 - 90s - loss: 2.0466 - accuracy: 0.2913 - val_loss: 2.1556 - val_accuracy: 0.2492 - 90s/epoch - 152ms/step
Epoch 3/5
591/591 - 90s - loss: 1.8532 - accuracy: 0.3555 - val_loss: 2.2200 - val_accuracy: 0.2702 - 90s/epoch - 153ms/step
Epoch 4/5
591/591 - 90s - loss: 1.7265 - accuracy: 0.4152 - val_loss: 2.1333 - val_accuracy: 0.3112 - 90s/epoch - 152ms/step
Epoch 5/5
591/591 - 90s - loss: 1.5849 - accuracy: 0.4763 - val_loss: 2.0193 - val_accuracy: 0.3633 - 90s/epoch - 152ms/step
